## 181 - Factor (categorical) variables in regression

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

from pygam import LinearGAM, s, l
from pygam.datasets import wage


import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
house = pd.read_csv('house_sales.csv', sep='\t')
print(house.PropertyType.unique())

['Multiplex' 'Single Family' 'Townhouse']


### Dummy Variables Representation

In [28]:
    # Before Dummies:
print(house.PropertyType.head())

1        Multiplex
2    Single Family
3    Single Family
4    Single Family
5    Single Family
Name: PropertyType, dtype: object


In [32]:
    # After Dummies:
    # By default, returns one hot encoding of the categorical variable.
    # The keyword argument drop_first will return P – 1 columns. Use this to avoid the problem of multicollinearity.
    # The method "get_dummies" takes the optional keyword argument drop_first to exclude the first factor as reference

print(pd.get_dummies(house['PropertyType'], drop_first=True).head(6))

   Single Family  Townhouse
1          False      False
2           True      False
3           True      False
4           True      False
5           True      False
6          False       True


In [36]:
predictors = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms',
              'BldgGrade', 'PropertyType']
outcome = 'AdjSalePrice'


    # If house[predictors] contains both categorical and numerical columns:
    #    - Numerical Columns: These are passed through as-is, unchanged
    #    - Categorical Columns: Dummy variables are created, with one less column than the number of categories 
    #      for each column (because of drop_first=True).

X = pd.get_dummies(house[predictors], drop_first=True)

house_lm_factor = LinearRegression()
house_lm_factor.fit(X, house[outcome])

print(f'Intercept: {house_lm_factor.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(X.columns, house_lm_factor.coef_):
    print(f' {name}: {coef}')


# There is no coefficient of Multiplex since it is implicitly defined when 
# "PropertyTypeSingle Family == 0" and "PropertyTypeTownhouse == 0". 
# The coefficients are interpreted as RELATIVE to Multiplex, 
# so a home that is Single Family is worth almost $85,000 less, and a home that is Townhouse is worth over $150,000 less.


Intercept: -446841.366
Coefficients:
 SqFtTotLiving: 223.37362892503822
 SqFtLot: -0.0703679813681255
 Bathrooms: -15979.013473415183
 Bedrooms: -50889.73218483028
 BldgGrade: 109416.3051614618
 PropertyType_Single Family: -84678.21629549256
 PropertyType_Townhouse: -115121.97921609186


In [ ]:
# Intercept:

    # The intercept is the predicted AdjSalePrice when all predictors (including dummy variables) are set to 0.
    # It may not always have a meaningful real-world interpretation, especially if setting all predictors to 0 doesn't make sense 
    # for your dataset (e.g., no square footage or 0 bathrooms). However, it's still a baseline value in the equation.

# Numerical Predictors:

# SqFtTotLiving: 223.3736
    # For every additional square foot of living space, the predicted price increases by approximately $223.37, 
    # holding all other variables constant.

# SqFtLot: -0.0704
    # For every additional square foot of lot size, the predicted price decreases by approximately $0.07, holding other variables constant.
    # This small negative coefficient might indicate that the lot size isn't strongly positively correlated with price, 
    # or its effect might be overshadowed by other factors.

# Bathrooms: -15979.01
    # Each additional bathroom decreases the predicted price by approximately $15,979, holding other variables constant.
    # This negative value is counterintuitive—it might suggest a correlation issue, multicollinearity, 
    # or that bathrooms are inversely associated with price in this specific dataset. Investigating this further is recommended.

# Bedrooms: -50889.73
    # Each additional bedroom decreases the predicted price by approximately $50,889, holding other variables constant.
    # This also seems counterintuitive. It might suggest that more bedrooms are associated with smaller or lower-quality homes, 
    # or there might be interaction effects with other variables.

# BldgGrade: 109416.31
    # For each increase in the building grade (likely a categorical measure of quality), the predicted price increases 
    # by approximately $109,416, holding other variables constant.
    # This aligns with intuition: higher-grade buildings are more valuable.


#Categorical Predictors (Dummy Variables):

# These coefficients represent the impact of a specific category relative to the omitted base category 
# (due to drop_first=True in pd.get_dummies).

# Base Category for PropertyType: "Multiplex"
    # Since the Multiplex category is omitted, its effect is included in the intercept. 
    # Other categories are interpreted relative to Multiplex.

# PropertyType "Single Family": -84678.22
    # Single Family homes are predicted to be approximately $84,678 less expensive than Multiplex properties, 
    # holding other variables constant.

# PropertyType_Townhouse: -115121.98
    # Townhouses are predicted to be approximately $115,122 less expensive than Multiplex properties, holding other variables constant.



# Next Steps

# Check for Multicollinearity: 
    # The negative coefficients for Bathrooms and Bedrooms might result from high correlation with SqFtTotLiving or other predictors. 
    # Use techniques like the Variance Inflation Factor (VIF) to investigate.

# Model Fit: 
    # Evaluate the model's overall performance (e.g., R2R2, RMSE) to understand how well it explains the data.

# Outliers and Nonlinear Effects: 
    # Review the dataset for outliers or potential nonlinear relationships that might affect coefficients.
    


In [40]:
X.head()

,SqFtTotLiving,SqFtLot,Bathrooms,Bedrooms,BldgGrade,PropertyType_Single Family,PropertyType_Townhouse
1,2400,9373,3.00,6,7,False,False
2,3764,20156,3.75,4,10,True,False
3,2060,26036,1.75,4,8,True,False
4,3200,8618,3.75,5,7,True,False
5,1720,8620,1.75,4,7,True,False


### Factor Variables with many levels


In [44]:
# The following line: 

# house['ZipCode']:
    # Accesses the ZipCode column of the house DataFrame.

# .value_counts():
    # Counts the occurrences of each unique value in the ZipCode column.
    # The result is a pandas Series, where the index contains unique ZipCode values, 
    # and the corresponding values are the counts of occurrences.

# pd.DataFrame(...):
    # Converts the resulting Series from .value_counts() into a DataFrame.
    # The ZipCode values become the DataFrame's index, and their counts become a single column 

# .transpose():
    # Transposes the resulting DataFrame.
    # Rows and columns are swapped, so:
    #     The original ZipCode values (index) become column names.
    #     The counts become a single row in the DataFrame.

print(pd.DataFrame(house['ZipCode'].value_counts()).transpose())

ZipCode  98038  98103  98042  98115  98117  98052  98034  98033  98059  98074  \
count      788    671    641    620    619    614    575    517    513    502   

ZipCode  ...  98051  98024  98354  98050  98057  98288  98224  98068  98113  \
count    ...     32     31      9      7      4      4      3      1      1   

ZipCode  98043  
count        1  

[1 rows x 80 columns]


#### Grouping ZIP codes

The code consolidates the 80 zip codes into five groups based on the median of the residual from the house_lm regression.

The median residual is computed for each zip, and the ntile function is used to split the zip codes, sorted by the median, into five groups.


In [100]:
# The code first builds a linear regression model to predict house prices:
    # This doesn't include ZipCode

predictors = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 
              'Bedrooms', 'BldgGrade']
outcome = 'AdjSalePrice'

house_lm = LinearRegression()
house_lm.fit(house[predictors], house[outcome])


# Then, it calculates residuals for each house:
    # A residual is the difference between the actual price and the predicted price. 
    # If a house's residual is positive, it sold for more than the model expected based on its features. 
    # This often indicates something about the location (zip code) is making houses more valuable.

    # The individual 'ZipCode' and 'residual' pairs are stored in the "zip_residuals" dataframe

zip_residuals = pd.DataFrame({
    'ZipCode': house['ZipCode'],
    'residual' : house[outcome] - house_lm.predict(house[predictors]),
})


# This calculates the median residual for each zip code. 
    # Think of this as measuring how much more or less expensive houses in each zip code are 
    # compared to what we'd expect based on their features alone. 
    # Some zip codes will consistently have positive residuals (more expensive than expected), 
    # others negative (less expensive than expected).

# This is a list comprehension, which is creating dictionary objects.
    # Basic list comprehension: 
    #     [expression for item in iterable]
    # In this case, the expression is a dictionary.
    # The iterable comes from "zip_residuals.groupby('ZipCode')", which creates groups of houses that share the same zip code. 
    # When we loop through these groups using for zipCode, x in, we get:
    #    - zipCode: the zip code value
    #    - x: a subset of the data containing all houses in that zip code

    # It's creating a list of dictionaries, where each dictionary has information about one zip code. 
    # Then pandas converts this list of dictionaries into a DataFrame, where each dictionary becomes a row.
    # It's like making a spreadsheet: each dictionary is like a row, 
    # and the dictionary keys ('ZipCode', 'count', 'median_residual') become the column headers. 

zip_groups = pd.DataFrame([
    {
        'ZipCode': zipCode,
        'count': len(x),
        'median_residual': x.residual.median()
    } 
    for zipCode, x in zip_residuals.groupby('ZipCode')
]).sort_values('median_residual')


# The final grouping happens here:
    # Rather than just splitting zip codes into 5 equal groups, 
    # it ensures each group contains roughly the same number of houses (not zip codes):
    #
    #    - It first sorts zip codes by their median residual (done when we created 'zip_groups')
    #    - 'cum_count' creates a running (cumulative) total across zip codes
    #       The cumulative count keeps adding up as we go through the zip codes, 
    #       which is why it's perfect for the subsequent qcut operation that divides the houses into five equal-sized groups.
    #    - 'qcut' then splits these running totals into 5 groups with equal numbers of houses

    # So, we sort all the zip codes according to their residual, then run a "cumulative counter",
    # in order to be able to cut this sorted list in groups with equal amount of houses, not postcodes.

zip_groups['cum_count'] = np.cumsum(zip_groups['count'])


    # 'qcut' then splits these running totals into 5 groups with equal numbers of houses
    # and creates a new column holding the group number 
    
# pd.qcut() is used to divide a continuous variable into quantiles (bins that each contain roughly the same number of data points).
    # zip_groups['cum_count']: the column being split into quantiles.
    # 5: The number of quantiles (bins)
    # "labels=False':  If False, the function assigns integer codes (0, 1, 2, ..., n-1) 
    #                  to represent the bins instead of creating categorical labels ("Bin 1", "Bin 2")
    #                  Without this, the result would default to a categorical data type with labels like "(0.0, 10.0]" for each bin
    # "retbins=False": If False, the function does not return the actual bin edges (cut points).
    #                  If set to True, pd.qcut() would return a tuple: the binned data and the array of bin edges.

zip_groups['ZipGroup'] = pd.qcut(zip_groups['cum_count'], 5, labels=False, retbins=False)
zip_groups.head()
print(zip_groups.ZipGroup.value_counts().sort_index())

# This approach is clever because it ensures each group represents about 20% of all houses, 
# even though some zip codes have many more houses than others. 
# The groups effectively represent "value tiers" of neighborhoods, from relatively undervalued (Group 0) 
# to relatively overvalued (Group 4), after accounting for house characteristics.


ZipGroup
0    16
1    16
2    16
3    16
4    16
Name: count, dtype: int64


In [104]:
# Extracting Relevant Columns:
# Extracts the ZipCode and ZipGroup columns from the zip_groups DataFrame.
# .set_index('ZipCode'): Sets "ZipCode" as the index of the resulting DataFrame (to_join).
#                        This makes ZipCode the key that will be used for matching during the join operation.
# The result: 'to_join' is a DataFrame where:
#    - The index is ZipCode.
#    - The column is ZipGroup.

to_join = zip_groups[['ZipCode', 'ZipGroup']].set_index('ZipCode')


# Joining 'to_join' with 'house'
# .join(to_join, on='ZipCode'): Joins the house DataFrame with to_join based on the ZipCode column in house and the index in to_join.
#                               This adds a new column (ZipGroup) to house, where the value is the corresponding ZipGroup for each ZipCode.
# The result: the house DataFrame now has a new column, 'ZipGroup'.

house = house.join(to_join, on='ZipCode')

# Converting ZipGroup to a Categorical Type:
# Converts the ZipGroup column in house to a categorical data type.
# Because categorical types are memory-efficient and useful when working with data that has a limited set of discrete values

house['ZipGroup'] = house['ZipGroup'].astype('category')

In [106]:
house

,DocumentDate,SalePrice,PropertyID,PropertyType,ym,zhvi_px,zhvi_idx,AdjSalePrice,NbrLivingUnits,SqFtLot,...,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ImpsVal,ZipCode,NewConstruction,ZipGroup
1,2014-09-16,280000,1000102,Multiplex,2014-09-01,405100,0.930836,300805.0,2,9373,...,6,7,1991,0,0,70000,229000,98002,False,2
2,2006-06-16,1000000,1200013,Single Family,2006-06-01,404400,0.929228,1076162.0,1,20156,...,4,10,2005,0,0,203000,590000,98166,True,2
3,2007-01-29,745000,1200019,Single Family,2007-01-01,425600,0.977941,761805.0,1,26036,...,4,8,1947,0,0,183000,275000,98166,False,2
4,2008-02-25,425000,2800016,Single Family,2008-02-01,418400,0.961397,442065.0,1,8618,...,5,7,1966,0,0,104000,229000,98168,False,2
5,2013-03-29,240000,2800024,Single Family,2013-03-01,351600,0.807904,297065.0,1,8620,...,4,7,1948,0,0,104000,205000,98168,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27057,2011-04-08,325000,9842300710,Single Family,2011-04-01,318700,0.732307,443803.0,1,5468,...,3,7,1951,0,0,201000,172000,98126,False,3
27058,2007-09-28,1580000,9845500010,Single Family,2007-09-01,433500,0.996094,1586196.0,1,23914,...,4,11,2000,0,1,703000,951000,98040,False,4
27061,2012-07-09,165000,9899200010,Single Family,2012-07-01,325300,0.747472,220744.0,1,11170,...,4,6,1971,0,0,92000,130000,98055,False,0
27062,2006-05-26,315000,9900000355,Single Family,2006-05-01,400600,0.920496,342207.0,1,6223,...,3,7,1939,0,0,103000,212000,98166,False,2
